In [1]:
import sympy as sp
import time
from sympy import Derivative as D
from utils.symbolic import (infinitesimals_generator, higher_infinitesimals_generator,
                            group_operator, der_relabel, deriv_infts, subs_new_vars, Diff,
                           sym_det_eqn)
from utils.algebra import (str_to_dict, get_common_factors, get_det_eqns, str_eqn_to_dict_eqn, 
                            simplify_redundant_eqn)

In [2]:
x, y, z, t = sp.symbols('x y z t')
phi = sp.Function('phi')(x, y, z, t)
psi = sp.Function('psi')(x, y, z, t)
b = sp.Function('b')(x, y, z, t)
w = sp.Function('w')(x, y, z, t)
sigma, gamma = sp.symbols('sigma gamma')
list_indep = [x, y, z, t]
list_dep = [phi, psi, b, w]
list_var = list_indep + list_dep
list_cte = [sigma, gamma]
list_all = list_cte + list_indep + list_dep
F =  D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) + sigma*(D(psi, x)*(D(psi, x, x, y) + D(psi, y, y, y))-
        D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))) + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) - gamma*(D(b,x,x) + D(b, y, y))
F_rules_array = {D(psi, x, x, y): 
        - (sigma*D(psi, x))**(-1)*(D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) 
        + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) 
        - gamma*(D(b,x,x) + D(b, y, y)))
        +(D(psi, x))**(-1)*D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))
        - D(psi, y, y, y)}
order = 3
F.expand()

-gamma*Derivative(b(x, y, z, t), (x, 2)) - gamma*Derivative(b(x, y, z, t), (y, 2)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (y, 3)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (x, 2), y) - sigma*Derivative(psi(x, y, z, t), (x, 3))*Derivative(psi(x, y, z, t), y) - sigma*Derivative(psi(x, y, z, t), y)*Derivative(psi(x, y, z, t), (y, 2), x) + Derivative(b(x, y, z, t), t) + Derivative(b(x, y, z, t), x)*Derivative(phi(x, y, z, t), y) - Derivative(b(x, y, z, t), y)*Derivative(phi(x, y, z, t), x) + Derivative(psi(x, y, z, t), x)*Derivative(w(x, y, z, t), y) - Derivative(psi(x, y, z, t), y)*Derivative(w(x, y, z, t), x)

In [3]:
x, t = sp.symbols('x t')
u = sp.Function('u')(x, t)
v = sp.Function('v')(x, t)
list_indep = [x, t]
list_dep = [u, v]
list_var = list_indep + list_dep
list_cte = []
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - D(u,x,x) +D(v,t) + x*v
F_rules_array = {D(u,x,x):D(u,t)+D(v,t)+x*v}
order = 2
F.expand()

x*v(x, t) + Derivative(u(x, t), t) - Derivative(u(x, t), (x, 2)) + Derivative(v(x, t), t)

In [2]:
x, t = sp.symbols('x t')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_var = list_indep + list_dep
list_cte = []
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - D(u,x,x)
F_rules_array = {D(u,x,x):D(u,t)}
order = 2
F.expand()

Derivative(u(x, t), t) - Derivative(u(x, t), (x, 2))

In [3]:
t = time.time()
infts = infinitesimals_generator(list_indep, list_dep)
n_indep = len(list_indep)
n_dep = len(list_dep)
infts_ind = infts[0:n_indep]
infts_dep = infts[n_indep:(n_dep + n_indep )]
elapsed = time.time() - t
print(elapsed)

0.0009968280792236328


In [4]:
t = time.time()
inft_derivatives, dep_vars_derivatives = higher_infinitesimals_generator(infts_ind, infts_dep,
                                                                            order, list_indep, list_dep)
infts_dummy = [item for sublist in inft_derivatives for item in sublist]
dep_vars_derivatives = [item for sublist in dep_vars_derivatives for item in sublist]
infts = infts + infts_dummy
elapsed = time.time() - t
print(elapsed)

0.38596630096435547


In [5]:
t = time.time()
F, deriv_names = der_relabel(dep_vars_derivatives, F)

vars_and_derivatives = list_indep + list_dep + deriv_names
elapsed = time.time() - t
print(elapsed)

0.000997781753540039


In [6]:
t = time.time()
XF = group_operator(F, vars_and_derivatives, infts) # Es la que mas se tarda
elapsed = time.time() - t
print(elapsed)

1.497368574142456


In [7]:
t = time.time()
XF = subs_new_vars(dep_vars_derivatives, deriv_names, XF)
elapsed = time.time() - t
print(elapsed)

0.0019948482513427734


In [8]:
t = time.time()
XF = XF.xreplace(F_rules_array)
elapsed = time.time() - t
print(elapsed)

0.002990245819091797


In [9]:
t = time.time()
empty_det_eqn = get_common_factors(XF, list_dep, list_indep)
elapsed = time.time() - t
print(elapsed)

0.019945383071899414


In [10]:
t = time.time()
det_eqn = get_det_eqns(XF, empty_det_eqn)
elapsed = time.time() - t
print(elapsed)

0.0049860477447509766


In [11]:
det_eqn

{'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2': ['+Derivative(xit(x,t,u(x,t)),(u(x,t),2))'],
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)': ['+2*Derivative(xix(x,t,u(x,t)),u(x,t))',
  '+Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)',
  '+Subs(Derivative(xit(x,t,_xi),_xi,x),_xi,u(x,t))'],
 'Derivative(u(x,t),t)': ['+Subs(Derivative(xit(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,x)',
  '-Subs(Derivative(xit(x,_xi_2,u(x,t)),_xi_2),_xi_2,t)',
  '+2*Subs(Derivative(xix(_xi_1,t,u(x,t)),_xi_1),_xi_1,x)'],
 'Derivative(u(x,t),x)^3': ['+Derivative(xix(x,t,u(x,t)),(u(x,t),2))'],
 'Derivative(u(x,t),x)*Derivative(u(x,t),t,x)': ['+2*Derivative(xit(x,t,u(x,t)),u(x,t))'],
 'Derivative(u(x,t),t,x)': ['+2*Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1),_xi_1,x)'],
 'Derivative(u(x,t),x)^2': ['-Derivative(etau(x,t,u(x,t)),(u(x,t),2))',
  '+Subs(Derivative(xix(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)',
  '+Subs(Derivative(xix(x,t,_xi),_xi,x),_xi,u(x,t))'],
 'Derivative(u(x,t),x)': ['+Subs(Derivative(xix(_xi_

In [12]:
t = time.time()
det_eqn = str_eqn_to_dict_eqn(det_eqn, list_var, list_all)
elapsed = time.time() - t
print(elapsed)

['+Derivative(xit(x,t,u(x,t)),(u(x,t),2))']
['+2*Derivative(xix(x,t,u(x,t)),u(x,t))', '+Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)', '+Subs(Derivative(xit(x,t,_xi),_xi,x),_xi,u(x,t))']


SympifyError: Sympify of expression 'could not parse 'Derivative(xit(x, t, _xi), _xi, x), _xi, t, t)'' failed, because of exception being raised:
TokenError: ('EOF in multi-line statement', (2, 0))

In [ ]:
'Derivative(xit(x, t, _xi), _xi, x), _xi, t, t)'

In [13]:
t = time.time()
det_eqn = simplify_redundant_eqn(det_eqn)
elapsed = time.time() - t
print(elapsed)

0.0029914379119873047


In [14]:
t = time.time()
sym_det_eqn(det_eqn, list_indep, list_dep, list_cte)
elapsed = time.time() - t
print(elapsed)

0.03416037559509277


In [15]:
sym_det_eqn(det_eqn, list_indep, list_dep, list_cte)

Matrix([
[ 1,                                                                                                 Eq(xi^(t)_u_u, 0)],
[ 2,                                                                                                 Eq(xi^(t)_u_v, 0)],
[ 3,                                                                                  Eq(2*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[ 4,                                                                                                   Eq(xi^(t)_v, 0)],
[ 5,                                                                                     Eq(-xi^(x)_u + 2*xi^(x)_v, 0)],
[ 6,                                                                                                 Eq(xi^(x)_u_u, 0)],
[ 7,                                                                                                 Eq(xi^(x)_u_v, 0)],
[ 8,                                                                                      Eq(3*xi^(x)_u - xi^(x)_v, 0)],
[ 9,                   